<a href="https://colab.research.google.com/github/yosuke318/python_notebook/blob/main/dataclass_pydantic_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pprint

In [15]:
from dataclasses import dataclass
@dataclass(slots=True) # __slots__を使用することでメモリ使用量を大幅に削減できます
class Product:
  id: int
  name: str
  price:float

product1 = Product(id=1, name="apple", price=100.0)
pprint.pprint(product1)

product2 = Product(id="2", name="greenapple", price="1000")
pprint.pprint(product2)

Product(id=1, name='apple', price=100.0)
Product(id='2', name='greenapple', price='1000')


In [23]:
from pydantic.dataclasses import rebuild_dataclass
from pydantic import ValidationError, Field, BaseModel


class Book(BaseModel):
  title: str=Field(strict=True)
  pages: int=Field(strict=True)

try:
  book = Book(title="111", pages="123")
  pprint.pprint(book)
except ValidationError as e:
  pprint.pprint(e)

1 validation error for Book
pages
  Input should be a valid integer [type=int_type, input_value='123', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_type


In [31]:
from datetime import date
from typing import Optional, List
from pydantic import HttpUrl, Field

class TecnicalBook(Book):
  isbn: str = Field(strict=True, pattern=r'^(?:\d{10}|\d{13})$')
  author: str = Field(strict=True)
  publication_date: date = Field(strict=True)
  programming_language: Optional[str]=Field(default=None, strict=True)
  topics: List[str]=Field(default_factory=list)
  edition: Optional[int] = Field(defalut=1, strict=True, ge=1)
  publisher: str = Field(strict=True)
  website: Optional[HttpUrl] = Field(defalut=None)

  class Config:
    extra = 'forbid'

In [32]:
from datetime import date

tech_book = TecnicalBook(
    title="Python高速プログラミング",
    pages=300,
    isbn="9784873119120",
    author="ミーカ・ルーソ",
    publication_date=date(2023, 1, 1),
    programming_language="Python",
    topics=["最適化", "並列処理", "非同期プログラミング"],
    edition=2,
    publisher="オライリージャパン",
    website="https://www.oreilly.co.jp/books/9784873119120/"
)

pprint.pprint(tech_book)


TecnicalBook(title='Python高速プログラミング', pages=300, isbn='9784873119120', author='ミーカ・ルーソ', publication_date=datetime.date(2023, 1, 1), programming_language='Python', topics=['最適化', '並列処理', '非同期プログラミング'], edition=2, publisher='オライリージャパン', website=HttpUrl('https://www.oreilly.co.jp/books/9784873119120/'))
